In [4]:
from platform import python_version

print(python_version())

3.8.3


In [5]:
import numpy as np
# import functions_Full
# import importlib
# importlib.reload(functions_Full)
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import jitcdde as dde

In [6]:
abc = np.arange(20)
print(abc)
delay = 2
indices = np.sort(np.arange(-1,-len(abc),-delay))
abc_select = abc[indices]
print(abc_select)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[ 1  3  5  7  9 11 13 15 17 19]


In [19]:
from scipy.interpolate import interp1d
from jitcdde import jitcdde as dde
from jitcdde import y as y_dde
from jitcdde import t as t_dde
from sympy.utilities.lambdify import lambdify, implemented_function

print(type(y_dde))
print(type(t_dde))

f = lambdify(t_dde, t_dde, "tensorflow") 
print(type(f))
abc = f(t_dde)
print(type(abc))
bcd = y_dde(t_dde)

<class 'function'>
<class 'symengine.lib.symengine_wrapper.Symbol'>
<class 'function'>
<class 'symengine.lib.symengine_wrapper.Symbol'>


In [4]:
from jitcdde import jitcdde, y, t
import numpy as np

a = 4.8
b = 0.186
tau = 40

# Regular implementation
# ----------------------
# To implement the sine function, we use SymEngine’s sine. This is a symbolic function that gets translated to a C implementation of the sine function under the hood.

import symengine
f = [
		y(1),
		-a/tau*y(1) - b/tau*symengine.sin(y(0,t-tau)+y(1,t-tau))
	]
DDE_regular = jitcdde(f)

# With Callbacks
# --------------
# Now, let’s assume for example’s sake that we did not have a sine function for straightforward use like above (i.e., a symbolic SymEngine function that gets translated to a C implementation). Instead we have to use the one from the Python’s Math library.

my_sine = symengine.Function("my_sine")
inputs = [y(0,t-tau),y(1,t-tau)]
f_with_callback = [
        y(1),
        -a/tau*y(1) - b/tau*my_sine(*inputs)
    ]

# We need to introduce a wrapper to match the required signature for callbacks. We also add a `print` statement to see when the callback was called. Except for the latter, we do not use the first argument, which is a vector containing the entire present state of the system:

import math
def my_sine_callback(y,arg1,arg2):
    print(f"my_sine called with arguments {y} and {arg1} and {arg2}")
    return math.sin(arg1+arg2)

DDE_callback = jitcdde(
        f_with_callback,
        callback_functions = [(my_sine,my_sine_callback,2)],
    )

# Integration
# -----------
# Initialise and address initial discontinuities for both implementations:
DDE_regular.constant_past( [1.0,0.0], time=0.0 )
DDE_regular.adjust_diff()
DDE_callback.constant_past( [1.0,0.0], time=0.0 )
DDE_callback.adjust_diff()
assert DDE_regular.t == DDE_callback.t

# Integrate side by side and compare:
times = DDE_regular.t + np.arange(10,100,10)
for time in times:
	assert DDE_regular.integrate(time)[0] == DDE_callback.integrate(time)[0]

Generating, compiling, and loading C code.
Using default integration parameters.
Generating, compiling, and loading C code.
Using default integration parameters.
my_sine called with arguments [1. 0.] and 1.0 and 0.0
my_sine called with arguments [ 1.         -0.00195642] and 1.0 and 0.0
my_sine called with arguments [ 0.99853268 -0.00275855] and 1.0 and 0.0
my_sine called with arguments [ 0.99812184 -0.00368746] and 1.0 and 0.0
my_sine called with arguments [ 1.00000000e+00 -5.80283806e-04] and 1.0 and 0.0
my_sine called with arguments [ 9.99870914e-01 -8.54935357e-04] and 1.0 and 0.0
my_sine called with arguments [ 0.99982993 -0.00114016] and 1.0 and 0.0
my_sine called with arguments [ 1.00000000e+00 -2.54326749e-04] and 1.0 and 0.0
my_sine called with arguments [ 9.99975204e-01 -3.78514592e-04] and 1.0 and 0.0
my_sine called with arguments [ 9.99967110e-01 -5.04706753e-04] and 1.0 and 0.0
my_sine called with arguments [ 1.00000000e+00 -1.46759477e-04] and 1.0 and 0.0
my_sine called w